In [176]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import gudhi

In [306]:
def number_of_b0_b1(p):
    rips = gudhi.RipsComplex(points = p, max_edge_length = 2)
    s_tree = rips.create_simplex_tree(max_dimension = 2)
    diag = s_tree.persistence()
    b0, b1 = 0, 0
    for dg in diag:
        if(dg[1][1] >= 1.42 or dg[1][1] == np.inf):
            if(dg[0] == 0):
                b0 += 1
            if(dg[0] == 1):
                b1 += 1
    return b0, b1

In [305]:
def image_to_point(X):
    points = []
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            if(X[i, j] == 0):
                a = np.array([i, j])
                points.append(a)
    points = np.array(points)
    return points

In [177]:
img = cv2.imread('./images/simple_binary.png')

In [178]:
X = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
X = X[0:234, 0:234]

In [179]:
p_size = 80
patches_x = 3
patches_y = 3

In [180]:
X.shape

(234, 234)

In [181]:
offset = 0
X1 = np.zeros([X.shape[0], X.shape[1], 9]) + 255
X2 = np.zeros([X.shape[0], X.shape[1], 9]) + 255
X1_U_X2 = np.zeros([X.shape[0], X.shape[1], 9])

for i in range(0, X.shape[0], p_size):
    for j in range(0, X.shape[1], p_size):
        # print(f"px = {i+1}:{i+p_size}, py = {j+1}:{j+p_size}")
        X1[i+1:i+p_size-1, j+1:j+p_size-1, offset] = np.array(X[i+1:i+p_size-1, j+1:j+p_size-1])
        
        X2[:, :, offset] = X
        X2[i:i+p_size, j:j+p_size, offset] = 255
        
        X1_U_X2[:, :, offset] = X
        X1_U_X2[i:i+p_size, j:j+p_size+1, offset] = 255 
        X1_U_X2[i+1:i+p_size-1, j+1:j+p_size-1, offset] = X1[i+1:i+p_size-1, j+1:j+p_size-1, offset] 

        # cv2.imwrite(f"./images/croped/image_X1_{offset}.png", X1[:,:,offset])
        # cv2.imwrite(f"./images/croped/image_X2_{offset}.png", X2[:,:,offset])
        # cv2.imwrite(f"./images/croped/image_X1UX2_{offset}.png", X1_U_X2[:,:,offset])
        offset += 1